In [1]:
import soundfile as sf
import sounddevice as sd
import audio2numpy as a2n
import numpy as np
import math

In [9]:
## recording an audio file
duration= 5 # sec
fs= 16000 # sample rate
myrecord= sd.rec(int(duration * fs), samplerate= fs, channels= 1)
# print("Finised Recording")

In [53]:
myrecord[0]

array([0.], dtype=float32)

In [10]:
## checking weather recording finished or not
sd.wait()

In [45]:
## palying an audio file
sd.play(myrecord, samplerate= fs)

In [5]:
# myrecord2=[]
# for i in range(60):
#     myrecord2.append(myrecord[fs*i:fs*(i+1)])
myrecord2= [myrecord[fs*i:fs*(i+1)] for i in range(duration)]

In [52]:
len(myrecord2)
sd.play(myrecord2[3], samplerate=fs)
# print(myrecord2[10])

In [19]:
type(myrecord)
len(myrecord)

960000

In [74]:
fs= 16000
n= duration
myrecord1= myrecord[fs*n:fs*(n+1)]
sd.play(myrecord1)

In [221]:
path= 'F:\\Downloads\\Documents\\Cough Detector\\Segmented Flusense Data\\train\Cough\\0__-_5kbw2Mcw_cough-2.wav'
data, fs = sf.read(path)

In [224]:
sd.play(data, fs)

(44096,)

# Model 

In [2]:
# from IPython import display
# import librosa, librosa.display
# import matplotlib.pyplot as plt
# import numpy as np

import os
import shutil

from jupyter_client.manager import KernelManager
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
#
from skimage.transform import resize
import keras.backend as K
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import *
from keras.utils.vis_utils import plot_model
from keras import layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import *
from keras.activations import sigmoid, tanh
import random
from keras.metrics import *
from keras.regularizers import l2

import librosa

In [8]:
seed_ = 5
batch_size= 32

In [9]:
my_classes= ["Not_Cough", "Cough"]

In [10]:
# def load_wav_16k_mono(filename):
#     """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
#     file_contents = tf.io.read_file(filename)
#     wav, sample_rate = tf.audio.decode_wav(
#         file_contents,
#         desired_channels=1)
#     wav = tf.squeeze(wav, axis=-1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
#     return wav

def load_wav_16k_mono(audio_file, sr):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """ 
    wav= audio_file
    sample_rate= sr
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

# def load_wav_for_map(audio_file, sr, label):
#     return load_wav_16k_mono(audio_file, sr), label

In [11]:
data= np.reshape(data, (data.shape[0], 1))
data= tf.cast(data, dtype= tf.float32)


NameError: name 'data' is not defined

In [80]:
t_wav= load_wav_16k_mono(data, tf.cast(44100, dtype= tf.int32))

In [12]:
## applies the embedding extraction model to a wav data

def extract_embedding(wav_data, label):
    '''run YAMNet to extract embedding from the wav data'''
    scores, embeddings, spectrogram= yamnet_model(wav_data)
    num_embeddings= tf.shape(embeddings)[0]
    return (embeddings, tf.repeat(label, num_embeddings))

In [13]:
yamnet_model_path = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_path)

OSError: SavedModel file does not exist at: C:\Users\Asus\AppData\Local\Temp\tfhub_modules\9616fd04ec2360621642ef9455b84f4b668e219e\{saved_model.pbtxt|saved_model.pb}

In [17]:
input_segment= tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                        name='input_segment')

X= Dense(512, LeakyReLU(0.1))(input_segment)
X= Dense(256, LeakyReLU(0.1))(X)
output_layer= Dense(2)(X)

my_model= Model(input_segment, output_layer)
# my_model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(1024), dtype=tf.float32, #1024
#                           name='input_embedding'),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(len(my_classes))
# ], name='my_model')


my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer= tf.keras.optimizers.Adam(learning_rate= 1e-5),
                metrics=['accuracy'])
# my_model.compile()
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_segment (InputLayer)  [(None, 1024)]            0         
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 656,642
Trainable params: 656,642
Non-trainable params: 0
_________________________________________________________________


## Prediction

In [99]:
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
        file_contents,
        desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

def load_wav_for_map(filename, label):
    return load_wav_16k_mono(filename), label


def extract_embedding(wav_data, label):
    '''run YAMNet to extract embedding from the wav data'''
    scores, embeddings, spectrogram= yamnet_model(wav_data)
    num_embeddings= tf.shape(embeddings)[0]
    return (embeddings, tf.repeat(label, num_embeddings))

In [91]:
path= 'D:\Python Codes\Python Projects\Cough Detector Project\EZTMC4Z-coughing_mono.wav'
data, sr= sf.read(path)
sd.play(data)

du= len(data)/sr

In [64]:
tf.cast(sr, dtype= tf.int64)

<tf.Tensor: shape=(), dtype=int64, numpy=44100>

In [87]:
path= 'D:\Python Codes\Python Projects\Cough Detector Project\EZTMC4Z-coughing.wav'
testing_wav_data= load_wav_16k_mono(path) #(myrecord2)

duration= np.ceil(len(testing_wav_data) / 16000)

# sd.play(testing_wav_data)

In [88]:
weights_path= "D:\Python Codes\Python Projects\Cough Detector Project\Weights\\try4_ep_135_val_acc_0.874.h5" #"/content/drive/MyDrive/Trained_Weights/pretrained_Cough_detection_YAMNet/pretrained_Cough_detection_3/keras_metadata.pb"
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
# result = my_model(embeddings).numpy()
weights= my_model.load_weights(weights_path)
result= my_model.predict(embeddings)


inferred_class = my_classes[result.mean(axis=0).argmax()]
print(f'The main sound is: {inferred_class}')

The main sound is: Cough


Function for prediction

In [100]:
weights_path= "D:\Python Codes\Python Projects\Cough Detector Project\Weights\\try4_ep_135_val_acc_0.874.h5" #"/content/drive/MyDrive/Trained_Weights/pretrained_Cough_detection_YAMNet/pretrained_Cough_detection_3/keras_metadata.pb"
weights= my_model.load_weights(weights_path)
def do_prediction(testing_wav_data):
    scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
    result= my_model.predict(embeddings)
    inferred_class = my_classes[result.mean(axis=0).argmax()]
    print(f'The main sound is: {inferred_class}')
    return result.mean(axis=0).argmax()

do_prediction(t_wav)


The main sound is: Cough


1

In [18]:
scores, embeddings, spectrogram = yamnet_model(tf.squeeze(myrecord2[0]))
result= my_model.predict(embeddings)
inferred_class = my_classes[result.mean(axis=0).argmax()]
print(f'The main sound is: {inferred_class}')

The main sound is: Not_Cough


In [82]:
sd.play(t_wav)

In [106]:
testing_wav_data= t_wav
# duration= len(testing_wav_data)/
fs= 16000
s=2 ## do prediction over 2s audio
cr=0
for i in range(int(np.ceil(duration/s))):
    temp_wav_data= testing_wav_data[fs*i:fs*(i+s)]
    r= do_prediction(temp_wav_data)
    cr= cr+r

print(f"Cough rate is {cr/duration} coughs per sec")

The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
The main sound is: Cough
Cough rate is 0.5 coughs per sec


# Final Assemble

In [108]:
## Final assemble

def config_variable():
    fs= 16000 #smaple rate
    sec_per_audio= 2 ## audio length for giving to prediction
    my_classes= ["Not_Cough", "Cough"]



def load_mymodel_and_weights(weights_path):
    """ This function will load all the needed material for doing prediction like the whole model and variables"""
    
    
    
    """loading model"""
    
    yamnet_model_path = 'https://tfhub.dev/google/yamnet/1'
    yamnet_model = hub.load(yamnet_model_path)
    
    input_segment= tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                    name='input_segment')
    X= Dense(512, LeakyReLU(0.1))(input_segment)
    X= Dense(256, LeakyReLU(0.1))(X)
    output_layer= Dense(2)(X)
    my_model= Model(input_segment, output_layer)
    
    
    my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    optimizer= tf.keras.optimizers.Adam(learning_rate= 1e-5),
                    metrics=['accuracy'])
    
    weights= my_model.load_weights(weights_path)

## This function will predict wheather the given audio file have cough or not in it. 

def do_prediction(testing_wav_data):
    
    """Input= numpy audio file
        output 1 if the audio have cough, 0 if not
    """
    
    scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
    result= my_model.predict(embeddings)
    inferred_class = my_classes[result.mean(axis=0).argmax()]
    
    print(f'The main sound is: {inferred_class}')
    
    return result.mean(axis=0).argmax() ## output 1 if the audio have cough, 0 if not




In [109]:
def recording_audio(duration, fs=16000):
    """This function will record audio file for given sec"""
    recorded_audio= sd.rec(int(duration*fs), samplerate= fs, channels= 1)
    sd.wait()
    return recorded_audio


def load_wav_16k_mono(audio_file, sr):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """ 
    wav= tf.cast(audio_file, dtype= tf.float32)
    # sample_rate= sr
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sr, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

def load_wav_for_map(audio_file, sr, label):
    return load_wav_16k_mono(audio_file, sr), label


In [110]:
## load config
## load the model
## record the audio
## convert it to segments to predict
##


config_variable()


##### Hidden

In [181]:
## for mono channel audio file

# def mono_channel_audio(audio_file):
#     nchannels= audio_file.shape[1]
#     return np.reshape(audio_file.sum(axis= 1)/nchannels, (audio_file.shape[0],1))



# def load_wav_16k_mono(audio_file, sr= 16000):
#     """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
#     wav= audio_file
#     sample_rate= int(sr)
#     wav = tf.squeeze(wav, axis=-1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=tf.cast(16000, dtype= tf.int64))
#     return wav

# def load_wav_for_map(audio_file, sr= 16000, label):
#     return load_wav_16k_mono(audio_file, sr), label


## For loading file from internal storage

# def load_wav_16k_mono(filename):
#     """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
#     file_contents = tf.io.read_file(filename)
#     wav, sample_rate = tf.audio.decode_wav(
#         file_contents,
#         desired_channels=1)
#     wav = tf.squeeze(wav, axis=-1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
#     return wav

# def load_wav_for_map(audio_file, label):
#     return load_wav_16k_mono(audio_file), label


# def load_wav_16k_mono(audio_file, sr= 16000):
#     """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
#     # file_contents = tf.io.read_file(filename)
#     # wav, sample_rate = tf.audio.decode_wav(
#     #     file_contents,
#     #     desired_channels=1)
#     wav= audio_file
#     sample_rate= sr
#     wav = tf.squeeze(wav, axis=-1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
#     return wav

# def load_wav_for_map(test_audio_file, sr= 16000, label):
#     return load_wav_16k_mono(test_audio_file, sr), label




SyntaxError: non-default argument follows default argument (Temp/ipykernel_604/3282203217.py, line 52)

In [194]:

# Function for Cough rate
weights_path = "D:\Python Codes\Python Projects\Cough Detector Project\Weights\\try4_ep_135_val_acc_0.874.h5"
weights = my_model.load_weights(weights_path)

def cough_rate(test_audio_file, sr):
    # "/content/drive/MyDrive/Trained_Weights/pretrained_Cough_detection_YAMNet/pretrained_Cough_detection_3/keras_metadata.pb"
    
    testing_wav_data= load_wav_16k_mono(test_audio_file, sr)
    scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
    # result = my_model(embeddings).numpy()
    result = my_model.predict(embeddings)
    inferred_class = my_classes[result.mean(axis=0).argmax()]
    # print(f'The main sound is: {inferred_class}')
    return result.mean(axis=0).argmax()
    


In [253]:
path= "D:\Python Codes\Python Projects\Cough Detector Project\EZTMC4Z-coughing_mono.wav"
test_wav, sr= librosa.load(path) #channels= 2
# test_wav= test_wav.reshape((test_wav.shape[0],1))
test_wav.shape

(507456,)

In [254]:
sd.play(test_wav, samplerate= sr)

Trying to load audio data for the model

In [143]:
# test_wav, sr= librosa.load(path)
# len(test_wav), sr
path= "D:\Python Codes\Python Projects\Cough Detector Project\EZTMC4Z-coughing.wav"
test_wav, sr= sf.read(path) #channels= 2
# sd.play(test_wav, sr)
# duration= int(np.ceil(len(test_wav)/sr))
# myrecord[fs*i:fs*(i+1)] for i in range(duration)
# duration
nchannels= test_wav.shape[0]
test_wav= test_wav.sum(axis=1)/nchannels

# if test_wav.shape[1]== 2 :
#     test_wav= (test_wav[:,0]+test_wav[:,1])/2
temp= librosa.load(path, sr= 16000)
type(list(temp))

list

In [ ]:
cr= []
fs=16000 
for i in range(duration):
    cr.append(cough_rate(test_wav[fs*i: fs*(i+1)], sr))

cr= np.sum(cr)/duration
print(cr)

In [215]:
def mono_channel_audio(audio_file):
    nchannels= audio_file.shape[1]
    return np.reshape(audio_file.sum(axis= 1, keepdims= True)/nchannels, (audio_file.shape[0],1))

path= "D:\Python Codes\Python Projects\Cough Detector Project\EZTMC4Z-coughing.wav"
test_wav, sr= sf.read(path) #channels= 2
test_wav= mono_channel_audio(test_wav)
sf.write("D:\Python Codes\Python Projects\Cough Detector Project\EZTMC4Z-coughing_mono.wav", test_wav, samplerate= sr)
sd.play(test_wav, sr)

In [212]:
a= np.eye(3,4)
a=a.sum(axis=0, keepdims= True)
a.shape

(1, 4)